In [0]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate gradio

**Imports**

In [0]:
import os
import requests
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
import gradio as gr

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

**Model**

In [0]:
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
  model_name,
  device_map="auto",
  quantization_config=quant_config
)

**Tokenizer**

In [0]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

**Functions**

In [0]:
def generate_dataset(topic, number_of_data, inst1, resp1, inst2, resp2, inst3, resp3):
    # Convert user inputs into multi-shot examples
    multi_shot_examples = [
        {"instruction": inst1, "response": resp1},
        {"instruction": inst2, "response": resp2},
        {"instruction": inst3, "response": resp3}
    ]

    # System prompt
    system_prompt = f"""
    You are a helpful assistant whose main purpose is to generate datasets.
    Topic: {topic}
    Return the dataset in JSON format. Use examples with simple, fun, and easy-to-understand instructions for kids.
    Include the following examples: {multi_shot_examples}
    Return {number_of_data} examples each time.
    Do not repeat the provided examples.
    """

    # Example Messages
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Please generate my dataset for {topic}"}
    ]

    # Tokenize Input
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
    streamer = TextStreamer(tokenizer)

    # Generate Output
    outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

    # Decode and Return
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


def gradio_interface(topic, number_of_data, inst1, resp1, inst2, resp2, inst3, resp3):
    return generate_dataset(topic, number_of_data, inst1, resp1, inst2, resp2, inst3, resp3)

**Default Values**

In [0]:
default_topic = "Talking to a (5-8) years old and teaching them manners."
default_number_of_data = 10
default_multi_shot_examples = [
    {
        "instruction": "Why do I have to say please when I want something?",
        "response": "Because it’s like magic! It shows you’re nice, and people want to help you more."
    },
    {
        "instruction": "What should I say if someone gives me a toy?",
        "response": "You say, 'Thank you!' because it makes them happy you liked it."
    },
    {
        "instruction": "why should I listen to my parents?",
        "response": "Because parents want the best for you and they love you the most."
    }
]

**Init gradio**

In [0]:
gr_interface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(label="Topic", value=default_topic, lines=2),
        gr.Number(label="Number of Examples", value=default_number_of_data, precision=0),
        gr.Textbox(label="Instruction 1", value=default_multi_shot_examples[0]["instruction"]),
        gr.Textbox(label="Response 1", value=default_multi_shot_examples[0]["response"]),
        gr.Textbox(label="Instruction 2", value=default_multi_shot_examples[1]["instruction"]),
        gr.Textbox(label="Response 2", value=default_multi_shot_examples[1]["response"]),
        gr.Textbox(label="Instruction 3", value=default_multi_shot_examples[2]["instruction"]),
        gr.Textbox(label="Response 3", value=default_multi_shot_examples[2]["response"]),
    ],
    outputs=gr.Textbox(label="Generated Dataset")
)

**Run the app**

In [0]:
gr_interface.launch()